In [1]:
from dictionary_learning.evaluation import evaluate

import pandas as pd
import torch as t
from nnsight import LanguageModel
from dictionary_learning import ActivationBuffer, AutoEncoder
from dictionary_learning.trainers import StandardTrainer
from dictionary_learning.training import trainSAE
from dictionary_learning.utils import read_csv

In [2]:
ae = AutoEncoder.from_pretrained("/gpfs/helios/home/jpauklin/dictionary_learning/saes/estMedSaeX16/trainer_0/ae_1105.pt").to("cuda") # to is rquired to load to GPU

data = read_csv("/gpfs/space/projects/stacc_health/data-synthetic/100k_synthetic_texts.csv", 100_000)

/gpfs/helios/home/jpauklin/dictionary_learning/dictionary_learning/dictionary.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t.load(path)


In [3]:
device = "cuda:0" #GPU
# Load Model
model_name = "/gpfs/space/projects/stacc_health/gpt2_model/estMed-gpt2_fine_tuned4/estMed-gpt2_fine_tuned4"
model = LanguageModel(
    model_name,
    device_map=device,
)

activation_dim = model.transformer.h[0].ln_1.normalized_shape[0] # output dimension of the MLP = 768
submodule = model.transformer.h[11].mlp

buffer = ActivationBuffer( # buffer will yield batches of tensors of dimension = submodule's output dimension
    data=data,
    model=model,
    submodule=submodule,
    d_submodule=activation_dim, # output dimension of the model component
    n_ctxs=3e4,  # length of each context. you can set this higher or lower dependong on your available memory
    device=device,
    out_batch_size = 2048 # reduce batch size to limit memory usage.
) 

In [4]:
out = evaluate(dictionary = ae, 
               activations = buffer,
               device = device,
              )

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
"""
    MSE loss: average squared L2 distance between an activation and the autoencoder's reconstruction of it
    L1 loss: a measure of the autoencoder's sparsity
    L0: average number of features active above a random token
    Percentage of neurons alive: fraction of the dictionary features which are active on at least one token out of dictionary.dict_size random tokens
    
    CE diff: difference between the usual cross-entropy loss of the model for next token prediction and the cross entropy when replacing activations with our dictionary's reconstruction
    Percentage of CE loss recovered: when replacing the activation with the dictionary's reconstruction, the percentage of the model's cross-entropy loss on next token prediction that is recovered (relative to the baseline of zero ablating the activation)
"""

In [5]:
out

{'l2_loss': 5.859640121459961,
 'l1_loss': 20.421188354492188,
 'l0': 45.0107421875,
 'frac_variance_explained': 0.7675023078918457,
 'cossim': 0.8705556392669678,
 'l2_ratio': 0.7453358173370361,
 'relative_reconstruction_bias': 0.8966822624206543,
 'loss_original': 4.567016124725342,
 'loss_reconstructed': 4.580355644226074,
 'loss_zero': 4.4883832931518555,
 'frac_recovered': 1.1696431636810303,
 'frac_alive': 0.6669108271598816}